<a href="https://colab.research.google.com/github/ahsenozturk1/hi-work---veri-analizi/blob/main/Veri_analizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AQUASTAT VERİSETİNİ TANIMA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data = pd.read_csv("AQUASTAT Dissemination System.csv")
data.head()

In [ ]:
new_data = data.copy()
display(new_data.head())

In [ ]:
new_data.tail()

In [ ]:
new_data.info()

In [ ]:
new_data.isnull().any()

In [ ]:
new_data.describe()

## Adımlar:

1-Veriyi gözlemleme: head() ve tail() ile ilk ve son satırlara bakıldı.

2-Yapıyı incele: info() ile kolonları, veri tiplerini ve boş değerleri kontrol edildi.

3-Sayısal veriyi özetle: describe() ile toplam, ortalama, medyan, standart sapma ve min/max değerleri incelendi.

## 1-Fonksiyon:

In [ ]:
#Kolonlarda kaç farklı değer olduğu göterilir.
def show_unique_counts(data):
    """
    Her kolon için kaç farklı değer olduğunu ve tüm benzersiz değerleri eksiksiz gösterir.
    """

    for col in data.columns:
        unique_vals = data[col].unique()
        count_unique = data[col].nunique()
        print(f"Kolon: {col}")
        print(f"Farklı değer sayısı: {count_unique}")
        print(f"Değerler: {unique_vals}\n")

# Kullanımı
show_unique_counts(data)

## 2-Fonksiyon:

In [ ]:
def show_value_counts(data):
    """
    Her kolon için value_counts çıktısını DataFrame olarak döndürür.
    """
    value_counts_dfs = []
    for col in data.columns:
        print(f"Kolon: {col}") # Still print the column name for context
        value_counts_df = data[col].value_counts(dropna=False).reset_index()
        value_counts_df.columns = [col, 'count'] # Rename columns for clarity
        value_counts_dfs.append(value_counts_df)
    return value_counts_dfs

# Kullanımı
value_counts_tables = show_value_counts(data)

# Display each DataFrame in the list
for table in value_counts_tables:
    display(table)

## **Verisetinin Açıklaması:**
***************************************************
Kolon Sayısı: 10

Eksik Değer: Yok

Analiz Odaklı Kolonlar: Variable, Area, Year, Value

###**Temel Kolonlar**
*********************************************

**-Variable (17 gösterge):** Tarımsal, endüstriyel, belediye, toplam su çekimi; kişi başına su kullanımı; sulama, hayvancılık, enerji ve çevresel akış gibi göstergeler.

**-Area (59 ülke/bölge):** Hem ülke düzeyinde (Türkiye, Fransa, ABD vb.) hem de kıtasal/toplam bölgeler (Avrupa, Kuzey Afrika, Doğu Asya).

**-Year:** 2019–2022 dönemini kapsar.

**-Value:** Su kullanımı miktarları, oranları ve kişi başına düşen değerler (2419 farklı sayı).

### **Diğer Kolonlar**
*********************************************************************

**-Unit (3 ölçü birimi):**

10^9 m³/year: Yıllık toplam su kullanımı (milyar m³).

%: Toplam kullanım içindeki oran.

m³/inhab/year: Kişi başına yıllık su kullanımı (m³).

**-Symbol (Veri Türü):**

A (Official) → Resmî kaynaklardan gelen veriler.

I (Imputed) → Eksik verilerin tahminiyle doldurulmuş değerler.

E (Estimate) → Resmî veri yoksa uzman/modeller tarafından üretilen tahmini değerler.

X → Veri mevcut değil. (Bazı durumlarda Export (X) ihracatı ifade eder.)

**-IsAggregate:**

False → Tekil ülkeler

True → Kıta veya küresel toplulaştırılmış değerler




IsAggregate_True = Kıtalar

IsAggregate_False = Ülkeler

In [ ]:
non_aggregate_data = new_data[new_data['IsAggregate'] == False]
display(non_aggregate_data)

In [ ]:
aggregate_data = new_data[new_data['IsAggregate'] == True]
display(aggregate_data.head())

************************************************************************************************************************************************************
*********************************************************************
#**ÜLKELERİN ANALİZİ:**



## **İlk Filtreleme Adımı:Ülke Bazında Resmî (A) Su Çekimi Verilerinin Filtrelenmiş Hali (10^9 m³/Year)**

**Birim seçimi (Unit):**

Yalnızca 10^9 m³/year birimindeki satırlar alındı.

Böylece sadece yıllık toplam su çekim miktarları (milyar m³) analiz kapsamına girdi.

**Veri kaynağı (Symbol):**

'A' (Official) seçildi.

Yani sadece resmî kurumlar tarafından sağlanan güvenilir veriler
kullanıldı.
**IsAggregate:**
False → Ülke bazındaki gözlemler alındı.

In [ ]:
df_filtered = data[
    (data['Unit'] == '10^9 m3/year') &
    (data['Symbol'] == 'A' ) &
    (data['IsAggregate'] == False)
].drop(columns=['Symbol', 'IsAggregate', 'Unit','m49','VariableGroup','Subgroup'])

df_filtered.info()

In [ ]:
df_filtered['Area'].value_counts().reset_index()

In [ ]:
df_filtered['Variable'].value_counts().reset_index()

-'Environment Flow Requirements ' 1 değer olduğu icin bunu dahil etmiyoruz.

In [ ]:
exclude_vars = ['Environmental Flow Requirements']

# Filtrele
df_filtered = df_filtered[~df_filtered['Variable'].isin(exclude_vars)]

### Aralık Atama:
-'value'degerlerini örnegin '0.692' ya da '7.392' gibi degerleri bir aralıga atıyoruz '0-1','1-3','3-5','5+' seklinde.

In [ ]:
bins = [0, 1, 3, 5, df_filtered['Value'].max()]
labels = ['0-1', '1-3', '3-5', '5+']

df_filtered['Value_range'] = pd.cut(df_filtered['Value'], bins=bins, labels=labels, include_lowest=True)


In [ ]:
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head()

### Segment
-'Value_Range'sütununa göre segment ekliyoruz.

In [ ]:
# Segment ekleme (Value_range'e göre)
segment_map = {
    '0-1': 'Low',
    '1-3': 'Medium',
    '3-5': 'High',
    '5+': 'Very High'
}

df_filtered['Segment'] = df_filtered['Value_range'].map(segment_map)
df_filtered.head()

In [ ]:
pivot = df_filtered.pivot_table(index='Area', columns='Year', values='Segment', aggfunc='first')
print(pivot)

In [ ]:
pivot = df_filtered.pivot_table(index='Area', columns='Year', values='Segment', aggfunc='first')
print(pivot)


In [ ]:
# Tüm yılları ve ülkelerde veri olan satırları al
df_complete = df_filtered.dropna(subset=['Segment'])
df_complete.head(90)

## **Hangi Su Kullanım Türü Türkiye’de En Fazla:Çubuk Grafiği**
-"Variable" kolonu "Total water withdrawal" olmayan satırlar filtreleniyor.

-Her grup için "Value" kolonunun ortalaması alındı.
Sonuçlar büyükten küçüğe doğru sıralandı.

In [ ]:
# Türkiye'nin verilerini al
turkiye_all = df_complete[df_complete['Area'] == 'Türkiye']

# "Total water withdrawal" dışındakileri seç
turkiye_vars = turkiye_all[turkiye_all['Variable'] != 'Total water withdrawal']

# Değişken bazında ortalama al
turkiye_summary = turkiye_vars.groupby('Variable')['Value'].mean().sort_values(ascending=False)

print(turkiye_summary)

In [ ]:
keep_vars = [
    "Agricultural water withdrawal",
    "Industrial water withdrawal",
    "Municipal water withdrawal"
]

turkiye_filtered = turkiye_vars[turkiye_vars["Variable"].isin(keep_vars)]

# Ortalama değerleri hesapla
turkiye_summary = (
    turkiye_filtered.groupby("Variable")["Value"]
    .mean()
    .sort_values(ascending=False)
)

# Grafik çiz
plt.figure(figsize=(10,6))
sns.barplot(x=turkiye_summary.values, y=turkiye_summary.index, palette="viridis")
plt.title("Türkiye: Average Water Use (2019–2022)")
plt.xlabel("Value (10^9 m³/year)")
plt.ylabel("Variable")
plt.show()

## **En Yüksek Su Kullanımı: Sektör ve Ülke Bazında:Çubuk Grafiği**
-Bu grafik, her bir su kullanım türü (Variable) için en yüksek su çekim miktarını gösterir.

-Çubukların uzunluğu, Value (10^9 m³/year) cinsinden su kullanımını temsil eder.

-Her çubuğun yanında, bu maksimum değeri sağlayan ülke adı gösterilmiştir.

-Amaç: Hangi ülkelerin hangi su kullanım alanında en yüksek tüketimi gerçekleştirdiğini görsel olarak karşılaştırmak.

In [ ]:
max_values = []
variables = df_filtered['Variable'].unique()

for var in variables:
    var_data = df_filtered[df_filtered['Variable'] == var]
    if not var_data.empty:
        max_row = var_data.loc[var_data['Value'].idxmax()]
        max_values.append({
            'Variable': var,
            'Country': max_row['Area'],
            'Value': max_row['Value']
        })

max_df = pd.DataFrame(max_values)
# Gösterilecek sektörler
selected_vars = [
    'Municipal water withdrawal',
    'Industrial water withdrawal',
    'Agricultural water withdrawal'
]

# max_df’i filtrele
filtered_df = max_df[max_df['Variable'].isin(selected_vars)]

# Sıralama (yatay barplotta en yüksek üstte olacak)
filtered_df = filtered_df.sort_values('Value', ascending=True)

# Grafik
plt.figure(figsize=(10,5))
sns.barplot(x='Value', y='Variable', data=filtered_df, palette='coolwarm')

# Ülke isimlerini barın yanında göster
for i, row in enumerate(filtered_df.itertuples()):
    plt.text(row.Value * 1.01, i, row.Country, va='center', fontsize=10)

plt.xlabel('Water Usage (10^9 m³/year)')
plt.ylabel('Variable')
plt.title('Highest Water Consumption: Selected Sectors')
plt.tight_layout()
plt.show()


## **İlk Analiz Sonucu:**
İlk analizimizde yalnızca resmî kaynaklı (A) veriler kullanıldı. Ancak bazı yıllara ait resmi veriler eksikti (NaN), bu da birçok ülkenin analiz dışında kalmasına yol açtı. Bu nedenle, analizimizi symbol filtresini kaldırarak tüm veriler üzerinden tekrar yapmaya karar verdik.
************************************************************************************************************************************************************
************************************************************************************************************************************************************

## **İkinci Filtreleme Adımı:Ülke Bazında 10^9 m³/Yıl Birimindeki Su Çekimi**

In [ ]:
df_filtered_ = data[
    (data['Unit'] == '10^9 m3/year') &
    (data['IsAggregate'] == False)
].drop(columns=['Symbol', 'IsAggregate', 'Unit','m49','VariableGroup','Subgroup'])

df_filtered_.info()

In [ ]:
df_filtered_.reset_index(drop=True)

In [ ]:
df_filtered_['Variable'].value_counts()

## **"Variable" Filtreleme:**
Burada daha çok verisi olan ve anlaşılır olan sektör başlıklarını alıyoruz :

Agricultural water withdrawal: Tarımsal su çekimi

Industrial water withdrawal: Endüstriyel su çekimi

Municipal water withdrawal: Belediye (kentsel) su çekimi


In [ ]:
# Önemli ve okunaklı değişkenler
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal',

]

# Filtreleme
df_filtered_= df_filtered_[df_filtered_['Variable'].isin(selected_variables)]

Burada ülkeleri kıtalara bölmek adına bir sözlük oluşturduk:

In [ ]:
country_to_region = {
    # Europe
    "Albania": "Europe",
    "Finland": "Europe",
    "Ukraine": "Europe",
    "Serbia": "Europe",
    "Romania": "Europe",
    "Republic of Moldova": "Europe",
    "Poland": "Europe",
    "Netherlands (Kingdom of the)": "Europe",
    "Lithuania": "Europe",
    "Latvia": "Europe",
    "Ireland": "Europe",
    "Hungary": "Europe",
    "United Kingdom of Great Britain and Northern Ireland": "Europe",
    "Bulgaria": "Europe",
    "Sweden": "Europe",
    "Belarus": "Europe",
    "Germany": "Europe",
    "Croatia": "Europe",
    "Slovakia": "Europe",
    "Spain": "Europe",
    "Malta": "Europe",
    "Czechia": "Europe",
    "Denmark": "Europe",
    "Italy": "Europe",
      # burada sen karar verebilirsin, Asya da denebilir
    "Estonia": "Europe",
    "Austria": "Europe",
    "Switzerland": "Europe",
    "Slovenia": "Europe",
    "France": "Europe",
    "North Macedonia": "Europe",
    "Portugal": "Europe",
    "Greece": "Europe",
    "Luxembourg": "Europe",
    "Belgium": "Europe",
    "Russian Federation": "Europe",  # basitlik için Europe’a alıyoruz
    "Norway": "Europe",
    "Iceland": "Europe",
    "Montenegro": "Europe",
    "Monaco": "Europe",
    "Bosnia and Herzegovina": "Europe",
    "Andorra": "Europe",
    "Liechtenstein": "Europe",

    # Asia
    "Türkiye": "Asia",
    "Mongolia": "Asia",
    "Japan": "Asia",
    "Republic of Korea": "Asia",
    "Democratic People's Republic of Korea": "Asia",

    # Northern Africa
    "Tunisia": "Northern Africa",
    "Algeria": "Northern Africa",
    "Morocco": "Northern Africa",
    "Egypt": "Northern Africa",
    "Sudan": "Northern Africa",
    "Libya": "Northern Africa",

    # Northern America
    "United States of America": "Northern America",
    "Canada": "Northern America",
}

In [ ]:
df_filtered_["Region"] = df_filtered_["Area"].map(country_to_region)
df_filtered_.reset_index(drop=True)

## **Dünya Genelinde Yıllara Göre Toplam Su Çekimi (2019–2022):Çizgi Grafiği**

-Yıllara göre her su kullanım türünün toplam su çekimi miktarını görselleştirmek.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# örnek: dünya geneli toplam
df_grouped = df_filtered_.groupby(["Year","Variable"])["Value"].sum().reset_index()

plt.figure(figsize=(10,6))
for var in df_grouped["Variable"].unique():
    subset = df_grouped[df_grouped["Variable"] == var]
    plt.plot(subset["Year"], subset["Value"], marker="o", label=var)

plt.title("Total Water Withdrawal (2019–2022)")
plt.xlabel("Year")
plt.ylabel("Quantity")
plt.legend()
plt.show()

### Sonuç :
2019–2022 döneminde dünya genelinde su çekiminin en büyük payı tarımsal kullanıma aittir. Tarımı endüstri takip ederken, en düşük pay ise belediye/evsel kullanımdadır. Bu tablo, küresel ölçekte suyun büyük ölçüde tarımsal faaliyetlerde tüketildiğini ve su yönetiminde tarımsal verimliliğin kritik önem taşıdığını göstermektedir.

## **Her Ülke İçin Yıllara Göre Toplam Su Çekimi:Çizgi Grafiği**


In [ ]:
import plotly.express as px

# Variable sütununu toplam (ülke+year bazlı)
df_total = df_filtered_.groupby(["Year","Area","Region"])["Value"].sum().reset_index()

continents = df_total["Region"].unique()

for continent in continents:
    continent_df = df_total[df_total["Region"] == continent]

    fig = px.line(
        continent_df,
        x="Year",
        y="Value",
        color="Area",
        markers=True,
        title=f"{continent} countries' total water withdrawal (10^9 m³/year, 2019–2022)",
        hover_name="Area",  # Üzerine gelince ülke ismi gösterir
        color_discrete_sequence=px.colors.qualitative.Set2  # Daha anlaşılır renkler
    )

    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Total water withdrawal (10^9 m³/yıl)",
        legend_title="Country",
        width=1200,
        height=700
    )

    fig.show()


## **Ülke ve Sektör Bazında En Yüksek Su Çekimi (2019–2022):Çubuk Grafiği**
Bu grafikler, 2019–2022 döneminde farklı bölgelerdeki ülkelerin çeşitli su kullanım sektörlerindeki toplam su çekim miktarlarını göstermektedir. Y ekseni toplam su çekimini (10^9 m³) ifade ederken, X ekseni ülkeleri temsil eder. Her çubuk ülkeye özel renkle gösterilmiş olup, üzerine gelindiğinde ülke ve değer bilgisi görüntülenebilir.

In [ ]:
import plotly.express as px

# İlgilendiğimiz bölgeler
target_regions = ["Europe", "Asia", "Northern Africa", "Northern America"]

# 2019-2022 toplam
df_period = df_filtered_[df_filtered_['Year'].isin([2019,2020,2021,2022])]
df_period = df_period.groupby(['Region','Variable','Area'])['Value'].sum().reset_index()

# Sadece seçilen bölgeleri al
df_period = df_period[df_period['Region'].isin(target_regions)]

# Kıta / bölge renkleri
region_colors = {
    "Europe": "#1f77b4",         # mavi
    "Asia": "#ff7f0e",   # turuncu
    "Northern Africa": "#2ca02c",# yeşil
    "Northern America": "#d62728" # kırmızı
}

sectors = df_period['Variable'].unique()

for region in target_regions:
    region_df = df_period[df_period['Region']==region]

    for sector in sectors:
        sector_df = region_df[region_df['Variable']==sector].sort_values(by='Value', ascending=False)
        top6 = sector_df.head(6)

        fig = px.bar(
            top6,
            x='Area',
            y='Value',
            color_discrete_sequence=[region_colors.get(region, '#7f7f7f')],  # bölge rengi
            hover_data={'Variable':True,'Value':True},
            title=f"{region} - Highest Countries in {sector} (2019-2022)",
            width=900,
            height=400
        )

        fig.update_layout(
            yaxis_title="Total Water Withdrawal (10^9 m³)",
            xaxis_title="Country",
            showlegend=False
        )

        fig.show()


Buraya kadar, 10⁹ m³/yıl birimi ile ifade edilen veriler kullanılarak, her ülkenin farklı sektörlerdeki su çekimi miktarları analiz edilmiştir.
***************************************************************
***************************************************************

## **Üçüncü Filitreleme Adımı: Ülke Bazında m^3/inhab/year Birimindeki Su Çekimi**

In [ ]:
data_filtered_ = data[
    (data['Unit'] == 'm3/inhab/year') &
    (data['IsAggregate'] == False)
].drop(columns=['Symbol', 'IsAggregate', 'Unit','m49','VariableGroup','Subgroup'])

data_filtered_.info()

In [ ]:
data_filtered_.reset_index(drop=True)

In [ ]:
data_filtered_['Variable'].value_counts()

## **"Variable" Filtreleme:**
Burada ilgili sektör başlıklarını alıyoruz :

Municipal water withdrawal per capita: Kişi başı belediye (kentsel) su çekimi

Agricultural water withdrawal per capita: Kişi başı tarımsal su çekimi

Industrial water withdrawal per capita: Kişi başı endüstriyel su çekimi




In [ ]:
# Önemli ve okunaklı değişkenler
selected_variables1 = [
    'Municipal water withdrawal per capita (total population)',
    'Industrial water withdrawal per capita',
    'Agricultural water withdrawal per capita',
]

# Filtreleme
data_filtered_= data_filtered_[data_filtered_['Variable'].isin(selected_variables1)]

In [ ]:
data_filtered_["Region"] = data_filtered_["Area"].map(country_to_region)
data_filtered_.reset_index(drop=True)

## **Dünya Genelinde Yıllara Göre Toplam Kişi Başı Su Çekimi (2019–2022):Çizgi Grafiği**

* Amaç: Yıllara göre her su kullanım türünün toplam kişi başı su çekimi miktarını görselleştirmek.

* Genel Düzeyde Karşılaştırma:
 En yüksek kişi başına kullanım: Endüstriyel su çekimi (turuncu çizgi).2019–2022 arasında hep zirvede. Bu da dünya genelinde kişi başına düşen su kullanımında sanayinin tarım ve belediyeden daha baskın olduğunu gösteriyor.Orta seviyede: Tarımsal su çekimi (mavi çizgi).Sanayinin oldukça gerisinde, ama belediyeden çok daha yüksek.En düşük: Belediye su çekimi (yeşil çizgi).Kişi başına düşen en küçük kullanım hane halkı/şehir düzeyinde.

* Zaman İçindeki Değişimler (2019–2022)

  Sanayi (turuncu):
  2019–2020’de yatay seyir → 2021’de zirve yapıyor → 2022’de tekrar düşüyor.Pandemi etkisi olabilir: 2020’de üretim kısıtlıydı, 2021’de toparlanma ile sanayi su talebi arttı, 2022’de tekrar düşüş eğilimi başlamış.

  Tarım (mavi):
  Hafif bir azalış trendi var (2019 → 2020 düşüyor, sonra çok hafif toparlansa da genel olarak geriliyor). Bu, ya tarımda sulama verimliliğinin arttığını ya da nüfus artışıyla kişi başına su kullanımının azaldığını gösterebilir.

  Belediye (yeşil):
  Neredeyse düz bir çizgi → 2019–2022 arasında büyük bir değişim yok. Hane halkı su kullanımı istikrarlı kalmış.


In [ ]:
# örnek: dünya geneli toplam
data_grouped = data_filtered_.groupby(["Year","Variable"])["Value"].sum().reset_index()

plt.figure(figsize=(10,6))
for var in data_grouped["Variable"].unique():
    subset = data_grouped[data_grouped["Variable"] == var]
    plt.plot(subset["Year"], subset["Value"], marker="o", label=var)

plt.title("Total Water Extraction ((m^3/inhab/year),2019–2022)")
plt.xlabel("Year")
plt.ylabel("Quantity")
plt.legend()
plt.show()

## **Her Ülke İçin Yıllara Göre Toplam Kişi Başı Su Çekimi: Çizgi Grafiği**




- Amacı: Farklı bölgeler (Avrupa, Kuzey Amerika, Kuzey Afrika, Asya) ve bu bölgelerin ülkeleri arasında kişi başına su çekim miktarlarını karşılaştırmak.Hangi ülke/bölgenin daha fazla veya daha az su kullandığını görmek.

- Genel Çıkarımlar:
 Avrupa ülkelerinde su yönetimi görece dengeli ve verimli; tarım ve sanayi su tüketimleri modern tekniklerle optimize edilmiş.

  Kuzey Amerika’da su kullanımı yüksek ama düşüş eğilimi var. Snayi ve tarım su yoğun ama verimli kullanım ya da su tüketim politikalarıyla düşüşte.

  Kuzey Afrika’da su stresi çok belirgin: yüksek çekimlere rağmen nüfus artışı ve kıt kaynaklar nedeniyle kişi başına değer sürekli azalıyor.

  Asya’da dağılım çok heterojen: Türkiye yüksek tüketici, Japonya sabit, diğerleri daha düşük seviyelerde.

In [ ]:
import plotly.express as px

# Variable sütununu toplam (ülke+year bazlı)
data_total = data_filtered_.groupby(["Year","Area","Region"])["Value"].sum().reset_index()

continents = data_total["Region"].unique()

for continent in continents:
    continent_df = data_total[data_total["Region"] == continent]

    fig = px.line(
        continent_df,
        x="Year",
        y="Value",
        color="Area",
        markers=True,
        title=f"{continent} Countries' Total Water Withdrawals Per Capita((m3/inhab/year), 2019–2022)",
        hover_name="Area",  # Üzerine gelince ülke ismi gösterir
        color_discrete_sequence=px.colors.qualitative.Set2  # Daha anlaşılır renkler
    )

    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Totlal Water Withdrawals Per Capita ( m³/inhab/year)",
        legend_title="Country",
        width=1200,
        height=700
    )

    fig.show()


## **Ülke ve Sektör Bazında En Yüksek Kişi Başı Su Çekimi (2019–2022): Çubuk Grafiği**
Bu grafikler, 2019–2022 döneminde farklı bölgelerdeki ülkelerin çeşitli su kullanım sektörlerindeki kişi başı toplam su çekim miktarlarını göstermektedir. Y ekseni toplam kişi başı su çekimini (10^3 m³/inhab/year) ifade ederken, X ekseni ülkeleri temsil eder. Her çubuk ülkeye özel renkle gösterilmiş olup, üzerine gelindiğinde ülke ve değer bilgisi görüntülenebilir.

* Amaç:
  Bu grafiklerin amacı, bölgesel ve sektörel farklılıkları görselleştirerek hangi ülkelerin hangi alanlarda yüksek su tüketimine sahip olduğunu göstermek ve böylece su yönetimi stratejilerine ışık tutmaktır.
* Genel çıkarımlar:

  Avrupa
  Tarımsal kullanımda: Yunanistan açık ara önde → sulamaya dayalı tarımın yüksekliği. İspanya ve Portekiz de yüksek.

  Endüstriyel kullanımda: Karadağ çok dikkat çekici şekilde yüksek → küçük nüfusa rağmen yoğun sanayi su tüketimi.

  Belediye kullanımında: İzlanda ve İrlanda önde → kişi başına düşen şehirsel su tüketimi fazla.

  Asya

  Tarımsal kullanımda: Türkiye lider → tarım suya en bağımlı sektör. Japonya da yüksek ama daha verimli görünüyor.

  Endüstriyel kullanımda: Japonya ve Güney Kore önde → sanayi yoğun ülkeler. Türkiye ise düşük.

  Belediye kullanımında: Güney Kore ve Japonya yüksek → şehirleşme ve yaşam standartlarının etkisi.

  Kuzey Afrika

  Tarımsal kullanımda: Libya, Mısır ve Sudan önde → tarımın suya bağımlılığı çok yüksek.

  Endüstriyel kullanımda: Mısır ve Libya öne çıkıyor → sanayileşmenin yoğun olduğu yerler.

  Belediye kullanımında: Libya ve Mısır en yüksek → su altyapısı ve tüketim alışkanlıkları etkili.

  Kuzey Amerika

  Tarımsal kullanımda: ABD Kanada’ya göre çok daha yüksek.

  Endüstriyel kullanımda: Kanada önde, ardından ABD → yoğun endüstriyel üretim.

  Belediye kullanımında: ABD daha yüksek → nüfus yoğunluğu ve tüketim alışkanlıkları.

In [ ]:
import plotly.express as px

# İlgilendiğimiz bölgeler
target_regions = ["Europe", "Asia", "Northern Africa", "Northern America"]

# 2019-2022 toplam
data_period = data_filtered_[data_filtered_['Year'].isin([2019,2020,2021,2022])]
data_period = data_period.groupby(['Region','Variable','Area'])['Value'].sum().reset_index()

# Sadece seçilen bölgeleri al
data_period = data_period[data_period['Region'].isin(target_regions)]

# Kıta / bölge renkleri
region_colors = {
    "Europe": "#1f77b4",         # mavi
    "Asia": "#ff7f0e",   # turuncu
    "Northern Africa": "#2ca02c",# yeşil
    "Northern America": "#d62728" # kırmızı
}

sectors = data_period['Variable'].unique()

for region in target_regions:
    region_df = data_period[data_period['Region']==region]

    for sector in sectors:
        sector_df = region_df[region_df['Variable']==sector].sort_values(by='Value', ascending=False)
        top6 = sector_df.head(6)

        fig = px.bar(
            top6,
            x='Area',
            y='Value',
            color_discrete_sequence=[region_colors.get(region, '#7f7f7f')],  # bölge rengi
            hover_data={'Variable':True,'Value':True},
            title=f"{region} - Highest Countries in {sector} (2019-2022)",
            width=900,
            height=400
        )

        fig.update_layout(
            yaxis_title="Total Water Withdrawal ( m³/inhab/year)",
            xaxis_title="Country",
            showlegend=False
        )

        fig.show()


Buraya kadar, m³/inhab/year birimi ile ifade edilen veriler kullanılarak, her ülkenin farklı sektörlerdeki kişi başı su çekimi miktarları analiz edilmiştir.
***************************************************************
***************************************************************

# **KITA BAZINDA GENEL GRAFİKLER**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

aggregate_data = new_data[new_data['IsAggregate'] == True]
display(aggregate_data.head())

aggregate_data[aggregate_data['IsAggregate']==True]['Area'].unique()
print(aggregate_data[aggregate_data['IsAggregate'] == True]['Area'].unique())

In [ ]:
# Define new_data by copying the original data DataFrame
data = pd.read_csv("AQUASTAT Dissemination System.csv")
new_data = data.copy()

# Original code from the selected cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

aggregate_data = new_data[new_data['IsAggregate'] == True]
display(aggregate_data.head())

aggregate_data[aggregate_data['IsAggregate']==True]['Area'].unique()
print(aggregate_data[aggregate_data['IsAggregate'] == True]['Area'].unique())

In [ ]:
# Önemli ve okunaklı değişkenler
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]

# Filtreleme
aggregate_data= aggregate_data[aggregate_data['Variable'].isin(selected_variables)]

## **Her Kıtanın Tüm Yıllar Boyunca Toplam Su Kullanımının Çubuk Grafiği (Bar Chart) ve Sunburst Chart (Güneş Patlaması Grafiği) ile Gösterimi**

Bir kıtanın toplam su kullanımının ne kadarının belirli bir amaca (örneğin, sanayi, tarım veya sulama) ayrıldığını görmemize yardımcı olmaktadır. Bu gösterim, bir karşılaştırmadan ziyade tekil bir kıtanın yapısını anlamak için kullanılmıştır.
Grafik, Doğu Asya'nın su tüketimindeki devasa payının büyük ölçüde tarım sektöründen kaynaklandığını açıkça göstermektedir. Aynı şekilde, Avrupa veya Kuzey Amerika'da sanayi veya belediye kullanımının daha belirgin olduğu görülmektedir.
Kuzey Afrika da ise tarım ön plandadır.
Bu analiz, bir kıtanın su kaynakları üzerindeki ana baskının nereden geldiğini anlamak için de çok önemlidir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Filter aggregate_data for the selected variables
aggregate_data = new_data[new_data['IsAggregate'] == True]
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]
filtered_aggregate_data = aggregate_data[aggregate_data['Variable'].isin(selected_variables)]

# 2. Gruplama ve Toplama: Tüm yılların toplamını alarak kıta ve değişken bazında veriyi hazırlayın
# Bu, "kıta seviyesindeki tüm verilerin toplamı" anlamına gelir.
toplam_su_kullanimi = filtered_aggregate_data.groupby(['Area', 'Variable'])['Value'].sum().reset_index()

# 3. Görselleştirme

# Çubuk grafiği oluşturun
plt.figure(figsize=(15, 8))
sns.barplot(
    data=toplam_su_kullanimi,
    x='Area',
    y='Value',
    hue='Variable', # Değişkenlere göre renklendirme (gruplama)
    palette='Spectral'
)

# Grafiğe başlık ve eksen etiketleri ekleyin
plt.title('Kıtalara Göre Toplam Su Kullanımı (Seçilen 3 Sektör)', fontsize=16)
plt.xlabel('Kıta', fontsize=12)
plt.ylabel('Toplam Su Kullanımı (10^9 m³/yıl)', fontsize=12)
plt.legend(title='Su Kullanım Türü', loc='upper right')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Sunburst Chart (Güneş Patlaması Grafiği) ile Gösterimi** (Etkileşimli)



Bu grafik ile her bir dilimin üzerine geldiğinizde, o dilimin toplam içerisindeki değerini ve yüzdesini görebilirsiniz.
Örneğin, Doğu Asya'nın iç halkasına tıkladığınızda, grafiğin odak noktası sadece Asya olacaktır, bu da o kıtanın sektörel dağılımını daha yakından incelemenize olanak tanır.

In [ ]:
import pandas as pd
import plotly.express as px

# Veri yükleme ve sütun adlandırma
# NOT: Kullanıcının belirttiği güncel dosya adı kullanıldı.
df = pd.read_csv('AQUASTAT Dissemination System.csv')
df.columns = ['m49', 'VariableGroup', 'Subgroup', 'Variable', 'Area', 'Year', 'Value', 'Unit', 'Symbol', 'IsAggregate']

# Değişken Listesi (Percentage values)
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]

# Filter for selected variables and aggregate data (continents)
df_filtered_aggregate = df[
    df['Variable'].isin(selected_variables) &
    (df['IsAggregate'] == True)
].copy()


# Toplam su kullanımı verisini al (summing percentages here will give the total percentage for each continent)
total_water_use_aggregate = df_filtered_aggregate.groupby(['Area', 'Variable'])['Value'].sum().reset_index()

# Sunburst Chart oluşturma
fig = px.sunburst(
    total_water_use_aggregate,
    path=['Area', 'Variable'],
    values='Value',
    color='Variable',
    color_discrete_map={
        'Agricultural water withdrawal as % of total water withdrawal': '#6a9f53',
        'Industrial water withdrawal as % of total water withdrawal': '#b8770d',
        'Municipal water withdrawal as % of total withdrawal': '#256598'
    },
    title='Kıtalara ve Sektörlere Göre Toplam Su Kullanımı Dağılımı (%)' # Updated title
)

# GÖRSEL İYİLEŞTİRMELER
fig.update_layout(
    title_font_size=24, # Başlık boyutunu artırdık
    title_x=0.5,
    margin=dict(t=50, l=0, r=0, b=0), # Grafiği daha fazla alana yaymak için kenar boşluklarını azalttık
    width=800, # Genişliği artırdık
    height=800
)

# Plotly Sunburst grafiğinde eksen yazıları (xticks) yerine,
# dilimlerin üzerindeki etiketlerin boyutunu güncelleyebiliriz.
fig.update_traces(
    textfont_size=16, # Etiket yazı boyutunu biraz daha büyüttük
    insidetextorientation='radial'
)

fig.update_traces(textinfo="label+value")

## **Her Kıtanın Yıllık Su Tüketimi Değişiminin Çizgi Grafiği (Line Plot) ile Gösterimi**


Veride her yıl için ayrı bir değer olması sebebiyle, zaman içindeki değişim bir çizgi grafiği (line plot) ile gösterilmiştir. Bu gösterim, hangi kıtaların su tüketimini artırdığını veya azalttığını görmemize yardımcı olmaktadır.Buna göre kıtalardaki su tüketimi yıllara göre dikkat çekici bir farklılık göstermemektedir.

In [ ]:
# 2. Gruplama ve Toplama: Tüm yılların toplamını alarak kıta ve değişken bazında veriyi hazırlayın
# Bu, "kıta seviyesindeki tüm verilerin toplamı" anlamına gelir.

# Filter aggregate_data for the selected variables
aggregate_data = new_data[new_data['IsAggregate'] == True]
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]
filtered_aggregate_data = aggregate_data[aggregate_data['Variable'].isin(selected_variables)]

toplam_su_kullanimi = filtered_aggregate_data.groupby(['Area', 'Variable', 'Year'])['Value'].sum().reset_index()

# Çizgi grafiği oluşturma
plt.figure(figsize=(15, 8))
sns.lineplot(
    data=toplam_su_kullanimi,
    x='Year',
    y='Value',
    hue='Area', # Her kıta için farklı bir çizgi
    marker='o' # Veri noktalarını belirginleştirmek için
)

# Grafiğe başlık ve eksen etiketleri ekleme
plt.title('Kıtalara Göre Yıllara Göre Toplam Su Kullanımı (Seçilen 3 Sektör)', fontsize=16)
plt.xlabel('Yıl', fontsize=12)
plt.ylabel('Toplam Su Kullanımı (10^9 m³/yıl)', fontsize=12)

# Legend'ı dışarıya alma
plt.legend(title='Kıta', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.xticks(toplam_su_kullanimi['Year'].unique())
plt.tight_layout()
plt.show()

## **Her Kıtanın Su Kullanım Trendleriyle ilgili Yıllık Değişim Analizinin, Çizgi Grafiği (Line Plot) ile Gösterimi**

Şimdiye kadar yapılan analizler genellikle toplam değerlere odaklanmaktadır,
Bu analiz ile, her bir kıtanın farklı su kullanım türlerinde (tarım, sanayi, belediye vb.) zaman içinde nasıl bir değişim yaşadığı görselleştirilmiştir.

Bu grafik sayesinde, Doğu Asya'daki tarımsal su tüketiminin yıllar içindeki sabitliğini ve Avrupa'daki sanayi su tüketiminin ufak dalgalanmalarını tek bir bakışta görebilmekteyiz. Ayrıca 2019-2020 arası Doğu Asya da sanayi ve tarım için su kullanımında bir düşüş yaşanmış, sonraki yıllarda büyük bir dalgalanma bu alanlarda olmamıştır. Kuzey Afrika ise yıllar içinde her sektörde su tüketiminde ufak dalgalanmalar göstermiştir.

In [ ]:
import pandas as pd
import plotly.express as px

# Yıllık bazda, her bir değişkenin ve kıtanın toplam su kullanımını grupla
yillik_trend = aggregate_data.groupby(['Year', 'Area', 'Variable'])['Value'].sum().reset_index()

# Önemli ve okunaklı değişkenler listesi
selected_variables = [
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]

# Sadece seçilen değişkenleri filtrele
yillik_trend = yillik_trend[yillik_trend['Variable'].isin(selected_variables)]


# Çizgi grafiğini facet_col (sütun bazlı ayırma) ile oluştur
# Her "Area" (Kıta) için ayrı bir sütun/grafik oluşturulur
fig = px.line(
    yillik_trend,
    x='Year',
    y='Value',
    color='Variable',  # Her değişken için farklı bir renk
    line_dash='Variable', # Farklı çizgi stili ekle
    markers=True,
    facet_col='Area', # Her kıta için ayrı bir alt grafik oluştur
    facet_col_wrap=4, # Her satırda en fazla 4 grafik göster
    title='Kıtalara Göre Yıllık Su Kullanım Trendleri (2019-2022)'
)

# Alt grafik başlıklarını daha anlaşılır yap
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Eksen etiketlerini ve düzeni güncelle
fig.update_layout(
    xaxis_title='Yıl',
    yaxis_title='Toplam Su Kullanımı (10^9 m³/yıl)',
    xaxis={'tickmode': 'linear'}
)
fig.show()

## **Her Kıtanın Tarımsal Su Tüketimi (Agricultural water withdrawal) Verisinin, Çubuk Grafiği (Bar Chart) ile Gösterimi**

Bu analiz, her bir kıtanın tüm yıllar boyunca ne kadar tarımsal suyu
kullandığını bize göstermektedir.


Tarımsal su tüketiminde en çok pay Doğu Asya da iken en az pay Avrupa'ya ait.

In [ ]:
# Sadece tarımsal su tüketimi verisini filtreleme (Kıtalar için)
tarimsal_su_tuketimi = aggregate_data[aggregate_data['Variable'] == 'Agricultural water withdrawal']

# Kıtalara göre tarımsal su tüketimi toplamını hesaplama
toplam_tarimsal_su = tarimsal_su_tuketimi.groupby('Area')['Value'].sum().reset_index()

# Çubuk grafiği oluşturma
plt.figure(figsize=(12, 7))
sns.barplot(
    data=toplam_tarimsal_su,
    x='Area',
    y='Value',
    palette='Blues_d'
)

# Başlık ve etiketler
plt.title('Kıtalara Göre Toplam Tarımsal Su Tüketimi (2019-2022)', fontsize=16)
plt.xlabel('Kıta', fontsize=12)
plt.ylabel('Toplam Su Tüketimi (10^9 m³/yıl)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## **Her Kıtanın Sanayi Su Kullanımı (Industrial water withdrawal) Verisinin,  Sunburst Chart (Güneş Patlaması Grafiği) ile Gösterimi**

Sunburst chart, hiyerarşik veriyi dairesel bir şekilde görselleştirir. Bu dinamik gösterimde, merkezden dışa doğru, verilerin daha alt seviyeleri gösterilmektedir. Bu veri setinde, merkezde Variable (değişken), dış halkada ise Area (kıta) yer almaktadır. Bu gösterim, toplam sanayi su kullanımının kıtalara göre nasıl dağıldığını dinamik bir şekilde görmemizi sağlar. Buna göre Kuzey Amerika da sanayi su tüketimi yüksektir. Ama Kuzey Afrika da sanayi su tüketimi en düşük seviyededir.

In [ ]:
import pandas as pd
import plotly.express as px

# Sadece sanayi su tüketimi verilerini filtreleme
industrial_water = new_data[new_data['Variable'] == 'Industrial water withdrawal']

# Kıtalara göre toplam sanayi su tüketimini hesaplama
toplam_industrial = aggregate_data.groupby('Area')['Value'].sum().reset_index()

# Sonuçları ekrana yazdırın
print(toplam_industrial)

In [ ]:
!pip install plotly

In [ ]:
# Sunburst chart oluşturma
import plotly.express as px
import pandas as pd

# Sadece sanayi su tüketimi verilerini filtreleme
industrial_water = aggregate_data[aggregate_data['Variable'] == 'Industrial water withdrawal']

# Filter out rows where 'Value' is zero
industrial_water = industrial_water[industrial_water['Value'] > 0]

# Check the data before creating the sunburst chart
print("Industrial Water Data Info (after filtering zero values):")
industrial_water.info()
print("\nIndustrial Water Data Head (after filtering zero values):")
display(industrial_water.head())

fig = px.sunburst(
    industrial_water,
    path=['Variable', 'Area'], # Merkezden dışa doğru hiyerarşi
    values='Value',
    color='Value',
    color_continuous_scale='Viridis',
    title='Kıtalara Göre Sanayi Su Tüketimi Dağılımı (Sunburst Chart)'
)

fig.update_traces(textinfo="label+value")

## **Her Kıtanın Belediye Suyu Kullanım (Municipal water withdrawal) Verisinin,  Treemap (Etkileşimli Ağaç Haritası) ile Gösterimi**

Bu grafik, hiyerarşik verileri iç içe geçmiş dikdörtgenler kullanarak gösterir. Dolayısıyla, bu grafik ile hangi kıtaların sulama suyu ihtiyacında en baskın olduğu tek bir bakışta görülebilir. Her bir dikdörtgenin boyutu, temsil ettiği değerle doğru orantılıdır. Yani, en büyük belediye suyu kullanan kıta, en büyük dikdörtgenle gösterilir.
Buna göre, Doğu Asya, belediye su kullanımında grafiğin büyük bir kısmını kaplamaktadır.





In [ ]:
import pandas as pd
import plotly.express as px

# Sütun adlarını basitleştirme
# df.columns = ['m49', 'VariableGroup', 'Subgroup', 'Variable', 'Area', 'Year', 'Value', 'Unit', 'Symbol', 'IsAggregate']

# Sadece belediye suyu kullanımı verilerini filtreleme
municipal_data = aggregate_data[new_data['Variable'] == 'Municipal water withdrawal']

# Kıtalara göre toplam belediye suyu kullanımını hesaplama
toplam_belediye_suyu = municipal_data.groupby('Area')['Value'].sum().reset_index()

# Sonuçlara göz atın
print(toplam_belediye_suyu)

In [ ]:
import pandas as pd
import plotly.express as px

# Sadece belediye suyu kullanımı verilerini filtreleme
municipal_data = aggregate_data[aggregate_data['Variable'] == 'Municipal water withdrawal']

# Kıtalara göre toplam belediye suyu kullanımını hesaplama
toplam_belediye_suyu = municipal_data.groupby('Area')['Value'].sum().reset_index()

# Treemap oluşturma
fig = px.treemap(
    toplam_belediye_suyu,
    path=[px.Constant("Kıtalar"), 'Area'], # Hiyerarşi: Sabit "Kıtalar" kökü altında Kıta isimleri
    values='Value',
    color='Value',
    color_continuous_scale='Blues', # Mavi renk skalası
    title='Kıtalara Göre Toplam Belediye Suyu Kullanımı (2019-2022) - Treemap'
)

fig.update_layout(
    title_font_size=24,
    title_x=0.5,
    margin=dict(t=50, l=0, r=0, b=0)
)
fig.update_traces(textinfo="label+value")
fig.show()

# **Sonuç: Kıtasal Anlamda Su Kullanımı ve Sektörel Dağılımlara Dair Önemli Çıkarımlar**

1.   Tarım Sektörünün Baskınlığı: Analizlerimizde net bir şekilde görüldü ki, kıtalar genelinde su kullanımının en büyük bölümü tarımsal faaliyetler tarafından gerçekleştiriliyor. Özellikle Doğu Asya gibi bölgelerde, toplam su kullanımının büyük bir yüzdesi tarım için ayrılmış durumda.
2.   Coğrafi Eşitsizlik: Kıtaların toplam su kullanımı arasında önemli bir fark bulunmaktadır. Nüfusu ve tarımsal üretimi yüksek olan Doğu Asya kıtası, diğer kıtalara kıyasla açık ara en yüksek su kullanımına sahiptir. Bu, küresel su kaynakları üzerindeki baskının coğrafi olarak eşit dağılmadığını göstermektedir.
3.   Toplam Tüketim ve Kişi Başı Tüketim Farkı: Bir kıta toplamda daha az su kullanıyor gibi görünse de, nüfusu az olduğu için kişi başına düşen su kullanımı yüksek olabilir. Bu da su kaynaklarının kullanım verimliliği açısından daha derinlemesine bir değerlendirme yapılması gerektiğini gösterir.
4.   Sektörel Önceliklerin Farklılığı: Kıtadan kıtaya su kullanım türlerinin dağılımı farklılık göstermektedir. Sanayi, belediye, ve tarım gibi sektörler, bazı bölgelerde toplam kullanım içinde daha büyük bir paya sahipken, diğerlerinde oldukça düşüktür. Bu durum, her kıtanın kendi ekonomik yapısı ve öncelikleri doğrultusunda farklı su yönetimi stratejileri benimsemesi gerektiğini işaret eder.

Sonuç olarak, bu çıkarımlar, elimizdeki su yönetimi, çevresel sürdürülebilirlik ve küresel ekonomiye dair önemli bulguları bize vermektedir.

# **KITA BAZINDA 2022 YILI GRAFİKLERİ**

## **Genel Değişkenlerin Tanımlanması**

In [ ]:
# =============================================================================
# Grafiklerde kullanılan ortak değişkenleri ve fonksiyonları tanımlama
# =============================================================================

# Tüm kıtalar
continents = aggregate_data[aggregate_data['IsAggregate']==True]['Area'].unique()

continents_custom_order = ['Eastern Asia', 'Northern America', 'Europe', 'Northern Africa']

latest_year = aggregate_data['Year'].max()  # 2022 yılını seçmek için

continent_data = new_data[(new_data['IsAggregate']==True) & (new_data['Year']==2022)]  # Kıtalar ve 2022 yılını seçmek için

# 2022 yılındaki toplam su kullanımını seçmek için
total_water_withdrawal_2022 = aggregate_data[(aggregate_data['Year']==2022) &
                                             (aggregate_data['Variable']=='Total water withdrawal')]

# Renk paleti

# Kıtaların renk kodları
continent_colors = {
    'Eastern Asia':'#1F77B4',   # mavi
    'Europe':'#FF7F0E',         # turuncu
    'Northern Africa':'#2CA02C',# yeşil
    'Northern America':'#D62728'# kırmızı
}

# Sektör renkleri
colors_sectors = ['#6A4C93', '#FF6F61', '#1982C4', '#FFD166'] # Belediye - Tarim - Endustri - Sulama

colors_sectors_dict = ['#6A4C93',  # Belediye – mor ton
                  '#FF6F61',  # Tarım – mercan ton
                  '#1982C4',  # Endüstri – canlı mavi
                  '#FFD166']  # Sulama – sarı ton


total_color = '#8AC926'



## **Kıtaların Toplam Su Kullanım Oranı (Pie Chart)**
2022 yılına dair toplam su kullanım oranına baktığımızda Doğu Asya'nın 43.7lik bir oranla en fazla su kullanan kıta olduğunu görüyoruz. Onu 30.1lik bir oranla Kuzey Amerika izlemektedir. Avrupa 17.7lik bir paya sahipken en son sırada 8.5lik bir oranla Kuzey Afrika gelmektedir.

In [ ]:
# =============================================================================
#  Kıtaların Toplam Su Kullanım Oranı(2022) (Pie Chart)
# =============================================================================

labels = total_water_withdrawal_2022['Area']
sizes = total_water_withdrawal_2022['Value']
colors = [continent_colors.get(area, '#333333') for area in total_water_withdrawal_2022['Area']]
explode = [0.05]*len(labels)

plt.figure(figsize=(6, 4))
plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%', # 43.7%, 17.7%, vs. seklinde yazabilmek icin
    explode=explode,
    colors=colors
)
plt.title(f'Total Water Withdrawal of Continents ({latest_year})', fontsize=16, fontweight='bold', pad=20)
plt.axis('equal')
plt.show()



##  **Kıtalararası Su Kullanım Karşılaştırması (Bar Chart)**
   Bu grafik her kıtanın farklı sektörlerdeki su kullanım miktarlarını gösterir.
   Buna göre:
   
*   Doğu Asya tarımsal su kullanımında lider konumda
*   Kuzey Amerika endüstriyel su kullanımında öne çıkıyor
*   Avrupa en dengeli su kullanım profiline sahip
*   Kuzey Afrika sulama amaçlı su kullanımı en düşük seviyede  

In [ ]:
agg_data = new_data[new_data['IsAggregate'] == True]
year = 2022

check_df = []

for continent in agg_data['Area'].unique():
    agri_total = agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Agricultural water withdrawal') & (agg_data['Year']==year)]['Value'].iloc[0]
    irr = agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Irrigation water withdrawal') & (agg_data['Year']==year)]['Value'].iloc[0] if not agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Irrigation water withdrawal') & (agg_data['Year']==year)].empty else 0
    livestock = agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Water withdrawal for livestock (watering and cleaning)') & (agg_data['Year']==year)]['Value'].iloc[0] if not agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Water withdrawal for livestock (watering and cleaning)') & (agg_data['Year']==year)].empty else 0
    aquaculture = agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Water withdrawal for aquaculture') & (agg_data['Year']==year)]['Value'].iloc[0] if not agg_data[(agg_data['Area']==continent) & (agg_data['Variable']=='Water withdrawal for aquaculture') & (agg_data['Year']==year)].empty else 0

    sum_parts = irr + livestock + aquaculture
    overlap = agri_total - sum_parts

    check_df.append({
        'Continent': continent,
        'Agricultural_total': agri_total,
        'Sum_of_parts': sum_parts,
        'Overlap (Agricultural - parts)': overlap
    })

import pandas as pd
pd.DataFrame(check_df)


In [ ]:
# =============================================================================
#  Kıtalararası Su Kullanım Karşılaştırması(2022) (Bar Chart)
# =============================================================================

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Agricultural water withdrawal
agr_data = aggregate_data[aggregate_data['Variable'] == 'Agricultural water withdrawal']
bars1 = ax1.bar(agr_data['Area'], agr_data['Value'],
                color=[continent_colors[area] for area in agr_data['Area']], alpha=0.8)
ax1.set_title('Agricultural water withdrawal (2022)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Water Quantity (10⁹ m³/year)')
ax1.tick_params(axis='x', rotation=45)

# 2. Industrial water withdrawal
ind_data = aggregate_data[aggregate_data['Variable'] == 'Industrial water withdrawal']
bars2 = ax2.bar(ind_data['Area'], ind_data['Value'],
                color=[continent_colors[area] for area in ind_data['Area']], alpha=0.8)
ax2.set_title('Industrial water withdrawal (2022)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Water Quantity (10⁹ m³/year)')
ax2.tick_params(axis='x', rotation=45)

# 3. Municipal water withdrawal
mun_data = aggregate_data[aggregate_data['Variable'] == 'Municipal water withdrawal']
bars3 = ax3.bar(mun_data['Area'], mun_data['Value'],
                color=[continent_colors[area] for area in mun_data['Area']], alpha=0.8)
ax3.set_title('Municipal water withdrawal (2022)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Water Quantity (10⁹ m³/year)')
ax3.tick_params(axis='x', rotation=45)

# 4. Irrigation water withdrawal
total_data = aggregate_data[aggregate_data['Variable'] == 'Irrigation water withdrawal']
bars4 = ax4.bar(total_data['Area'], total_data['Value'],
                color=[continent_colors[area] for area in total_data['Area']], alpha=0.8)
ax4.set_title('Irrigation water withdrawal (2022)', fontsize=14, fontweight='bold')
ax4.set_ylabel('Water Quantity (10⁹ m³/year)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.suptitle('Continental Water Withdrawal - 2022',
             fontsize=16, fontweight='bold', y=1.02)

plt.show()

## **Sektörel Su Dağılımı ve Tahmini Kişi Başına Su Kullanımı (Bar Chart)**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
data_2022 = continent_data

pivot_data = data_2022.pivot_table(
    index='Area',
    columns='Variable',
    values='Value',
    aggfunc='sum'
)

# 1. Sektörel dağılım yüzdeleri
total_col = 'Total water withdrawal'
agr_col = 'Agricultural water withdrawal'
ind_col = 'Industrial water withdrawal'
mun_col = 'Municipal water withdrawal'
irr_col = 'Irrigation water withdrawal'

pivot_data['Agr_Pct'] = (pivot_data[agr_col] / pivot_data[total_col]) * 100
pivot_data['Ind_Pct'] = (pivot_data[ind_col] / pivot_data[total_col]) * 100
pivot_data['Mun_Pct'] = (pivot_data[mun_col] / pivot_data[total_col]) * 100
pivot_data['Irr_Pct'] = (pivot_data[irr_col] / pivot_data[total_col]) * 100


# Yüzdelik dağılım grafiği
width = 0.6
areas = pivot_data.index

colors = colors_sectors
labels = ['Agricultural', 'Industrial', 'Municipal', 'Irrigation']
values = ['Agr_Pct', 'Ind_Pct', 'Mun_Pct', 'Irr_Pct']

bottom = 0
for label, value, color in zip(labels, values, colors):
    ax1.bar(areas, pivot_data[value], width, bottom=bottom, label=label, color=color)
    bottom += pivot_data[value]


ax1.set_ylabel('Percent (%)')
ax1.set_title('Sectoral Water Withdrawal (%)', fontweight='bold')
ax1.legend()
ax1.tick_params(axis='x', rotation=45)

# 2. Kişi başına su kullanımı tahmini (nüfus verisi olmadığı için gösterimsel)
per_capita_estimate = pivot_data[total_col] / [1670, 747, 252, 369]  # Tahmini nüfus (milyon)
bars = ax2.bar(
    areas,
    per_capita_estimate,
    color=[continent_colors[area] for area in areas],
    alpha=0.8
)
ax2.set_ylabel('Water withdrawal (m³/inhab/year)')
ax2.set_title('Water Withdrawal Per Capita Estimate', fontweight='bold')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


   ## **Sektörel Su Dağılımı (Pie Chart)**
    Bu analiz hangi kıtanın hangi sektörde ne kadar su kullandığını gösterir

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

areas = aggregate_data['Area'].unique()

for i, area in enumerate(areas):
    # Her kıta için sektörel veri hazırla
    area_data = aggregate_data[aggregate_data['Area'] == area]

    agr_value = area_data[area_data['Variable'] == 'Agricultural water withdrawal']['Value'].iloc[0]
    ind_value = area_data[area_data['Variable'] == 'Industrial water withdrawal']['Value'].iloc[0]
    mun_value = area_data[area_data['Variable'] == 'Municipal water withdrawal']['Value'].iloc[0]
    irr_value = area_data[area_data['Variable'] == 'Irrigation water withdrawal']['Value'].iloc[0]

    values = [agr_value, ind_value, mun_value, irr_value]
    labels = ['Agricultural', 'Industrial', 'Municipal','Irrigation']

    # Pasta grafiği oluştur
    wedges, texts, autotexts = axes[i].pie(values, labels=labels, autopct='%1.1f%%',
                                           colors=colors_sectors, startangle=90)

    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontsize(12)

    for text in texts:
        text.set_fontsize(12)


    axes[i].set_title(f'{area}\nTotal: {sum(values):.1f} billion m³/year',
                     fontsize=12, fontweight='bold')

plt.suptitle('Continental Water Withdrawal - 2022',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


## **Kıtalararası Sektörel Su Tüketimi Dağılımı (Pie Chart)**
Yüzde 43.1 gibi bir payla Tarımsal Su Tüketimi en fazla olan tüketim biçimidir. Onu 28.1 ile endüstriyel tüketim izlemektedir.

In [ ]:
sector_variables = ['Agricultural', 'Industrial', 'Municipal','Irrigation']
sector_totals = []
sector_labels = []

for sector in sector_variables:
    sector_data = aggregate_data[aggregate_data['Variable'].str.contains(sector, case=False, na=False)]
    if not sector_data.empty:
        total_value = sector_data['Value'].sum()
        sector_totals.append(total_value)
        sector_labels.append(
            f'{sector}' if sector == 'Agricultural' else
            f'{sector}' if sector == 'Industrial' else
            f'{sector}' if sector == 'Municipal' else
            f'{sector}'
        )

plt.figure(figsize=(6, 6))
if sector_totals:
    colors = colors_sectors[:len(sector_totals)]
    wedges, texts, autotexts = plt.pie(sector_totals, labels=sector_labels,
                                       colors=colors, autopct='%1.1f%%', startangle=90)
    plt.title('Continental Sectoral Distribution', fontweight='bold')

    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontweight('bold')
plt.show()


## **Kıta Bazında Sektörel Su Kullanımı Dağılımı (Bar Chart)**



Doğu Asya ve Kuzey Afrika'da su kullanımının en çok tarım alanında olduğunu görmekteyiz. Yine bu iki kıtada endüstriyel ve belediye su kullanımının tarım alanına nazaran çok küçük bir yüzdeyi kapsadığını görmekteyiz. Buna karşılık Kuzey Amerika kıtasında en fazla endüstriyel alanda su kullanılmaktadır, onu çok küçük bir farkla tarım alanı izlemektedir(yüzde 37.5). Belediye su kullanımının bu kıtada en düşük olduğu görülmektedir. Avrupa kıtasında ise yüzde 44.1lik bir oranla en fazla endüstriyel alanda su kullanılmakla birlikte belediye su kullanımı ve tarımda su kullanımının arasındaki farkın diğer üç kıtaya nazaran en az olduğu gözlemlenmektedir.

In [ ]:
check_df = []

for continent in continents_custom_order:
    agri = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Agricultural water withdrawal') &
        (continent_data['Year'] == latest_year)
    ]['Value'].iloc[0]

    irr = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Irrigation water withdrawal') &
        (continent_data['Year'] == latest_year)
    ]['Value'].iloc[0]

    livestock = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Water withdrawal for livestock (watering and cleaning)') &
        (continent_data['Year'] == latest_year)
    ]['Value'].iloc[0]

    aqua = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Water withdrawal for aquaculture') &
        (continent_data['Year'] == latest_year)
    ]['Value'].iloc[0]

    check_df.append({
        "Continent": continent,
        "Agricultural_total": agri,
        "Irrigation": irr,
        "Livestock": livestock,
        "Aquaculture": aqua,
        "Check_sum": irr + livestock + aqua
    })

import pandas as pd
check_df = pd.DataFrame(check_df)
print(check_df)


In [ ]:
continent_data = new_data[new_data['IsAggregate'] == True]
latest_year = continent_data['Year'].max()

# Yüzde değişkenleri
percentage_variables = [
    'Agricultural water withdrawal as % of total water withdrawal',
    'Industrial water withdrawal as % of total water withdrawal',
    'Municipal water withdrawal as % of total withdrawal'
]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, continent in enumerate(continents_custom_order):
    # Agricultural %
    agri_data = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == percentage_variables[0]) &
        (continent_data['Year'] == latest_year)
    ]
    agri_pct = agri_data['Value'].iloc[0] if not agri_data.empty else 0

    # Industrial %
    ind_data = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == percentage_variables[1]) &
        (continent_data['Year'] == latest_year)
    ]
    ind_pct = ind_data['Value'].iloc[0] if not ind_data.empty else 0

    # Municipal %
    mun_data = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == percentage_variables[2]) &
        (continent_data['Year'] == latest_year)
    ]
    municipal_pct = mun_data['Value'].iloc[0] if not mun_data.empty else 0

    # Irrigation % (hesaplama)
    irr_withdrawal = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Irrigation water withdrawal') &
        (continent_data['Year'] == latest_year)
    ]
    total_withdrawal = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == 'Total water withdrawal') &
        (continent_data['Year'] == latest_year)
    ]

    irr_pct = (
        (irr_withdrawal['Value'].iloc[0] / total_withdrawal['Value'].iloc[0]) * 100
        if not irr_withdrawal.empty and not total_withdrawal.empty else 0
    )

    categories = ['Agricultural', 'Industrial', 'Municipal', 'Irrigation']
    values = [agri_pct, ind_pct, municipal_pct, irr_pct]

    bars = axes[i].bar(categories, values, color=colors_sectors, alpha=0.8,
                       edgecolor='black', linewidth=1)

    for bar, value in zip(bars, values):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                     f'{value:.1f}%', ha='center', va='bottom',
                     fontweight='bold', fontsize=10)

    axes[i].set_title(continent, fontsize=12, fontweight='bold', pad=15)
    axes[i].set_ylabel('Percent (%)', fontsize=10, fontweight='bold')
    axes[i].set_ylim(0, 100)
    axes[i].grid(axis='y', linestyle='--', alpha=0.3)

plt.suptitle(f'Continental Distribution of Sectoral Water Withdrawal ({latest_year})',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()


In [ ]:

# =============================================================================
# Kıta Bazında Sektörel Su Kullanımı Dağılımı(2022) (Bar Chart)
# =============================================================================

continent_data = new_data[new_data['IsAggregate'] == True]
latest_year = continent_data['Year'].max()

# Yüzde değişkenleri
percentage_variables = [
    'Agricultural water withdrawal as % of total water withdrawal',
    'Industrial water withdrawal as % of total water withdrawal',
    'Irrigation water withdrawal as % of total water withdrawal'
]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, continent in enumerate(continents_custom_order):
    # Her kıta için ayrı grafik
    agri_data = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == percentage_variables[0]) &
        (continent_data['Year'] == latest_year)
    ]

    ind_data = continent_data[
        (continent_data['Area'] == continent) &
        (continent_data['Variable'] == percentage_variables[1]) &
        (continent_data['Year'] == latest_year)
    ]

    irr_data = continent_data[
    (continent_data['Area'] == continent) &
    (continent_data['Variable'] == percentage_variables[2]) &
    (continent_data['Year'] == latest_year)
    ]

    agri_pct = agri_data['Value'].iloc[0] if not agri_data.empty else 0
    ind_pct = ind_data['Value'].iloc[0] if not ind_data.empty else 0
    municipal_pct = 100 - agri_pct - ind_pct  #
    irr_pct = irr_data['Value'].iloc[0] if not irr_data.empty else 0
    municipal_pct = 100 - agri_pct - ind_pct - irr_pct


    categories = ['Agricultural', 'Industrial', 'Municipal', 'Irrigation']
    values = [agri_pct, ind_pct, municipal_pct, irr_pct]
    # colors_sect = ['#2E8B57', '#4682B4', '#DC143C']  # Yeşil, Mavi, Kırmızı
    colors_sect = colors_sectors

    bars = axes[i].bar(categories, values, color=colors_sect, alpha=0.8,
                      edgecolor='black', linewidth=1)


    for bar, value in zip(bars, values):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{value:.1f}%', ha='center', va='bottom',
                    fontweight='bold', fontsize=10)

    axes[i].set_title(f'{[continent][0]}',
                     fontsize=12, fontweight='bold', pad=15)
    axes[i].set_ylabel('Percent (%)', fontsize=10, fontweight='bold')
    axes[i].set_ylim(0, 100)
    axes[i].grid(axis='y', linestyle='--', alpha=0.3)

plt.suptitle(f'Continental Distribution of Sectoral Water Withdrawal ({latest_year})',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## **Kıtalara Göre Kişi Başı Su Kullanımı (Bar chart)**
Bütün alanlarda (endüstri, tarım ve belediye) ve bu alanların toplamında kişibaşına düşen su miktarı en fazla Kuzey Amerika kıtasındadır.

In [ ]:
# =============================================================================
# Kıtalara Göre Kişi Başı Su Kullanımı(2022) (Bar chart)
# =============================================================================

continent_data = new_data[new_data['IsAggregate'] == True]
latest_year = continent_data['Year'].max()

per_capita_variables = [
    'Total water withdrawal per capita',
    'Agricultural water withdrawal per capita',
    'Industrial water withdrawal per capita',
    'Municipal water withdrawal per capita (total population)'
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, variable in enumerate(per_capita_variables):
    values = []
    labels = []
    colors = []

    for continent in continents_custom_order:
        data = continent_data[
            (continent_data['Area'] == continent) &
            (continent_data['Variable'] == variable) &
            (continent_data['Year'] == latest_year)
        ]

        if not data.empty:
            values.append(data['Value'].iloc[0])
            # labels.append(get_tr_labels([continent])[0])
            labels.append([continent][0])
            colors.append(continent_colors.get(continent, '#333333'))

    bars = axes[i].bar(labels, values, color=colors, alpha=0.8,
                      edgecolor='black', linewidth=1)

    for bar, value in zip(bars, values):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.005,
                    f'{value:.1f}', ha='center', va='bottom',
                    fontweight='bold', fontsize=10)

    # axes[i].set_title(get_tr_variable(variable), fontsize=12, fontweight='bold', pad=15)
        axes[i].set_title((variable), fontsize=12, fontweight='bold', pad=15)

    axes[i].set_ylabel('m3/inhab/year', fontsize=10, fontweight='bold')
    axes[i].grid(axis='y', linestyle='--', alpha=0.3)

plt.suptitle(f'Per Capita Water Use by Continents ({latest_year})',
             fontsize=16, fontweight='bold', y=1.005)
plt.tight_layout()
plt.show()

## **Su Kullanımı Karşılaştırması (Heatmap)**
2022 yılına ait tüm alanlarda ve toplamda yapılan su kullanımının gözler önüne serildiği bu heatmap'te çok çarpıcı sonuçlar görülmektedir. Buna göre en az su kullanımı endüstri alanındaki 6.2 milyar metreküp ile Kuzey Afrika'da gerçekleşmiştir. En fazla su kullanımı da 437.1 milyar metreküp ile tarım alanında Doğu Asya'da olmuştur. Bu rakam Kuzey Afrika'nın endüstri için kullandığı suyun yaklaşık 70.5 katıdır!

In [ ]:
# =============================================================================
# Su Kullanımı Karşılaştırması(2022) (Heatmap)
# =============================================================================

continent_data = new_data[new_data['IsAggregate'] == True]
latest_year = continent_data['Year'].max()

key_variables = [
    'Total water withdrawal',
    'Agricultural water withdrawal',
    'Industrial water withdrawal',
    'Municipal water withdrawal'
]

# X ekseni labelları için Türkçe çeviri
# x_labels = [get_tr_variable(var) for var in key_variables]
x_labels = [(var) for var in key_variables]

# x_labels_short = ['Toplam', 'Tarımsal', 'Endüstriyel', 'Belediye']
x_labels_short = ['Total', 'Agricultural', 'Industrial', 'Municipal']


# Create matrix
heatmap_data = []
for continent in continents_custom_order:
    row = []
    for variable in key_variables:
        subset = continent_data[
            (continent_data['Area'] == continent) &
            (continent_data['Variable'] == variable) &
            (continent_data['Year'] == latest_year)
        ]
        value = subset['Value'].iloc[0] if not subset.empty else 0
        row.append(value)
    heatmap_data.append(row)

# Plot heatmap
plt.figure(figsize=(10, 6))
plt.xticks(rotation=0)  # Yatay etiketler

sns.heatmap(heatmap_data,
           xticklabels=x_labels_short,
          #  yticklabels=[get_tr_labels([c])[0] for c in continents_custom_order], # Türkçe kita isimleri
           yticklabels=[([c])[0] for c in continents_custom_order], # Türkçe kita isimleri

           annot=True, fmt='.1f', cmap='Blues')
plt.title(f'Water Withdrawal Comparison ({latest_year})', fontsize=16, fontweight='bold', pad=10)
plt.xlabel('Water withdrawal sector', labelpad=10, fontsize=10, fontweight='bold')  # X ekseni başlığı
plt.ylabel('Continents', labelpad=10, fontsize=10, fontweight='bold')
plt.tight_layout()
plt.show()

## **Tarımsal Su Kullanımı (Bar Chart)**

In [ ]:
agr_data = continent_data[continent_data['Variable'].str.contains('Agricultural', case=False, na=False)]

plt.figure(figsize=(7, 5))
if not agr_data.empty:
    agr_by_continent = agr_data.groupby('Area')['Value'].sum().sort_values(ascending=True)

    bars = plt.barh(range(len(agr_by_continent)), agr_by_continent.values,
                    color=[continent_colors.get(area, '#95a5a6') for area in agr_by_continent.index])

    plt.title('Agricultural Water Withdrawal', fontweight='bold')
    plt.xlabel('Water quantity (1 billion/year)')
    plt.yticks(range(len(agr_by_continent)), [area[:12] for area in agr_by_continent.index])

    for i, value in enumerate(agr_by_continent.values):
        plt.text(value + max(agr_by_continent.values) * 0.01, i,
                 f'{value:.1f}', va='center', fontweight='bold')
plt.show()


## **Belediye Su Kullanımı (Bubble Chart)**
Bu kabarcık grafiği üç veriyi aynı anda karşılaştırmamıza yardımcı oluyor: Toplam su harcamasının kişibaşına düşen miktarı y ekseninde yer alırken 2022 belediye su kullanımı x ekseninde gösterilmektedir. Kabarcıkların boyutu da ilgili kıtanın 2022 yılı toplam su kullanımını göstermektedir. Buradan görmekteyiz ki her ne kadar Doğu Asya toplam su kullanımı ve belediye su kullanımı konusunda zirvede olsa da bunun kişibaşına düşen miktarı en fazla Kuzey Amerika kıtasındadır. Bir diğer ilginç nokta da Kuzey Afrika diğer 3 kıtadan daha az belediye ve toplam su harcamış olsa da toplam suyun kişibaşına en fazla düştüğü ikinci kıtadır.

In [ ]:
# =============================================================================
# Kıtalar Bazında Belediye Su Kullanımı (2022) (Kabarcık Grafiği)(Bubble Chart)
# =============================================================================

continent_data = new_data[(new_data['IsAggregate']==True) & (new_data['Year']==2022)]

municipal_values = []
per_capita_values = []
bubble_sizes = []
labels = []
colors = []

for cont in continents_custom_order:
    total = continent_data[(continent_data['Area']==cont) &
                           (continent_data['Variable']=='Total water withdrawal')]['Value'].iloc[0]
    per_capita = continent_data[(continent_data['Area']==cont) &
                                (continent_data['Variable']=='Total water withdrawal per capita')]['Value'].iloc[0]
    municipal = continent_data[(continent_data['Area']==cont) &
                                (continent_data['Variable']=='Municipal water withdrawal')]['Value'].iloc[0]

    # Bubble boyutu: toplam su kullanımı (ölçekli)
    bubble_size = total * 10

    municipal_values.append(municipal)
    per_capita_values.append(per_capita)
    bubble_sizes.append(bubble_size)
    # labels.append(get_tr_labels([cont])[0])
    labels.append([cont][0])
    colors.append(continent_colors.get(cont, '#333333'))

# Bubble chart
plt.figure(figsize=(12,8))
plt.scatter(municipal_values, per_capita_values, s=bubble_sizes,
            c=colors, alpha=0.7, edgecolors='black', linewidth=1.5)

plt.margins(x=0.1, y=0.1)
plt.text(0.98, 0.98, 'Bubble size = Total water withdrawal (10^9 m³/year)',
         transform=plt.gca().transAxes, fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8),
         verticalalignment='top',
         horizontalalignment='right')

# Etiketler
for i, label in enumerate(labels):
    plt.annotate(label, (municipal_values[i], per_capita_values[i]),
                 xytext=(10,10), textcoords='offset points',
                 fontsize=11, fontweight='bold',
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.xlabel('Municipal Water Withdrawal (10^9 m³/year)', fontsize=12, fontweight='bold')
plt.ylabel('Total water withdrawal per capita (m³/inhab/year)', fontsize=12, fontweight='bold')
plt.title('Municipal Water Withdrawal by Continents in 2022', fontsize=16, fontweight='bold', pad=20)
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()


## **Yıllara Göre Su Kullanımı (Line Chart)**
Bu slope chart ile 2019 ve 2022 yılları arasında kullanılan toplam su miktarının çok ufak değişiklikler gösterdiğini görmekteyiz. En bariz gözüken değişim 2019 ve 2020 yılları arasında Doğu Asya'daki toplam su kullanımının azalmasıdır. Onun dışındaki tüm toplam su kullanımlarının oldukça stabil olduğu gözlemlenmektedir.

In [ ]:
print(aggregate_data['Variable'].unique())



In [ ]:
# =============================================================================
# Kıtalar Bazında Yıllara Göre Su Kullanımı
# =============================================================================
subset = aggregate_data[
    (aggregate_data['Area'].isin(continents)) &
    (aggregate_data['Variable'].isin([
        'Agricultural water withdrawal',
        'Industrial water withdrawal',
        'Irrigation water withdrawal',
        'Municipal water withdrawal'
    ]))
]

# Toplam su kullanımı
subset_total = subset.groupby(['Area','Year'])['Value'].sum().reset_index()

# UI iyilestirmesi - Grafikteki sira ile bilgilendirmedeki siranin eslesmesi icin
continent_final_values = {}
for area in continents:
    temp = subset_total[subset_total['Area']==area].sort_values('Year')
    continent_final_values[area] = temp['Value'].iloc[-1]

continents_sorted = sorted(continents, key=lambda x: continent_final_values[x], reverse=True)

plt.figure(figsize=(6,4))

for area in continents_sorted:
    temp = subset_total[subset_total['Area']==area].sort_values('Year')
    plt.plot(temp['Year'], temp['Value'], marker='o',
            label=[area][0],
            color=continent_colors.get(area, '#333333'), linewidth=2)

plt.xlabel('Year', fontsize=10, fontweight='bold', labelpad=10)
plt.ylabel('Total water withdrawal (1 billion/year', fontsize=10, fontweight='bold', labelpad=10)
plt.title('Water Use by Continents over the Year', fontsize=16, fontweight='bold', pad=10)
plt.xticks(temp['Year'].unique())
plt.grid(axis='y', linestyle='--', alpha=0.5)
legend = plt.legend(title='Continents', loc='upper left', bbox_to_anchor=(1, 1.025))
legend.get_title().set_fontweight('bold')

plt.show()